In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

from utils import get_trial_dataframe
from utils import get_trial_paths


In [53]:
# Replace with the path to the logs directory in multi-memory-task
logs_path = '/Users/aidapiccato/multi-memory-task/logs' 

# Replace this with the name of the dataset directories
datasets = ['2023_06_12_13_05_37']

trial_paths = get_trial_paths.get_trial_paths(logs_path=logs_path, datasets=datasets)
df = get_trial_dataframe.get_trial_dataframe(trial_paths=trial_paths)


Number of trials:  4


In [54]:
df

,trial_num,time,object_0_x,object_0_y,object_0_id,object_0_theta,object_1_x,object_1_y,object_1_id,object_1_theta,...,target_theta,num_object,response_object_ind,response_x,response_id,response_theta,correct,response_y,visible_s,delay_s
0,0,1.686590e+09,0.846499,0.549382,f,1.429233,0.153063,0.546206,a,-1.438392,...,1.429233,6,0.0,0.822796,f,1.423057,True,0.548040,1.532937,1.016625
2,2,1.686590e+09,0.242458,0.737007,a,-0.826897,0.825982,0.627419,d,1.198179,...,-0.826897,3,0.0,0.218764,a,-0.879590,True,0.732685,2.001913,0.497174
3,3,1.686590e+09,0.773440,0.718473,a,0.896678,0.445153,0.845676,d,-0.157353,...,0.896678,5,0.0,0.723078,a,0.871692,True,0.687554,0.501101,1.498854
